# 🚀 Local LLM Tutorial: Streaming Text Generation

## 📝 Abstract

This tutorial demonstrates how to implement streaming text generation using local language models.
We'll explore the Hugging Face Transformers library's pipeline abstraction, focusing on its
components, streaming capabilities, and generation parameters. The tutorial covers both the
practical implementation and the underlying mechanisms that make streaming generation possible.

## 🌟 Introduction

Language models have become increasingly powerful, but their practical deployment often requires
efficient handling of text generation, especially in interactive applications. Streaming generation
is crucial for creating responsive user experiences, as it allows for real-time display of
generated text rather than waiting for the entire output.

In this tutorial, we'll:

1. Set up a local language model using the Hugging Face pipeline
2. Implement streaming text generation
3. Configure generation parameters for optimal output
4. Understand the underlying components and their interactions

We'll use a suitable language model available on the Hugging Face Hub. The concepts and
techniques covered are applicable to most transformer-based language models.

**Note**: Some models (such as Mistral models) on the Hugging Face Hub require accepting terms of use (gated models).
To use these models, you'll need to:
  - Log in to your Hugging Face account (or create one).
  - Accept the terms on the model's page.
  - Set your Hugging Face API token as the `HF_TOKEN` variable in a `.env` file in the project root.

The tutorial assumes basic familiarity with Python and machine learning concepts, but we'll
explain each component in detail to ensure understanding of both the implementation and the
underlying mechanisms.

In [ ]:
from threading import Thread

import torch
import dotenv
from transformers import pipeline
from transformers import TextIteratorStreamer

dotenv.load_dotenv('.env')

# --- Choose a Model --- #
model_id = 'deepcogito/cogito-v1-preview-llama-3B'
#model_id = 'HuggingFaceTB/SmolLM2-1.7B-Instruct'
#model_id = 'mistralai/Mistral-7B-Instruct-v0.3'
#model_id = 'deepcogito/cogito-v1-preview-llama-3B'

## 🔍 Understanding the Pipeline

The Hugging Face pipeline is a high-level abstraction that combines several components:

### 🧠 Model
- Core language model (in this case, a causal language model)
- Uses transformer architecture
- Contains the model weights and architecture definition
- Handles the forward pass through the network

### 🔤 Tokenizer
- Converts between text and token IDs
- Splits text into tokens (subwords or words)
- Handles special tokens (BOS, EOS, padding)
- Manages vocabulary and token mappings

### ⚙️ Preprocessor
- Prepares inputs for the model
- Applies chat templates
- Handles padding and truncation
- Manages attention masks

### 🛠️ Postprocessor
- Processes model outputs
- Decodes token IDs back to text
- Handles special token removal
- Manages output formatting

The pipeline orchestrates these components in sequence:
```
Input Text → Tokenizer → Preprocessor → Model → Postprocessor → Output Text
```

For text generation specifically, the pipeline also handles:
- 📊 Generation strategies (greedy, sampling, beam search)
- 📏 Length control (min/max length, early stopping)
- 🌡️ Temperature and top-k/p sampling
- 🔄 Repetition penalties

When we create a pipeline with `device_map="auto"`, it also:
1. Loads the model weights
2. Determines available hardware (CPU/GPU)
3. Splits the model across devices if needed
4. Optimizes memory usage with the specified dtype (bfloat16)

## 📥 Download the Model (Optional but Recommended)

While the Transformers library will automatically download model files when initializing the pipeline,
using the HuggingFace CLI tool offers several advantages:

- **Faster downloads**: Uses optimized downloading with multiple connections
- **Better error handling**: Clearer feedback and retry mechanisms
- **Explicit caching**: More control over where and how models are stored
- **Progress tracking**: Visual feedback on download progress
- **Offline usage**: Download once, use anywhere without internet connection

The CLI command below is optional but recommended for a better experience:

In [ ]:
!huggingface-cli download --repo-type model {model_id}

## 1️⃣ Model Loading

First, we load the model using the Hugging Face pipeline. The pipeline provides a high-level
interface for text generation tasks. Key parameters:

- `model`: The model identifier from Hugging Face Hub (selected above)
- `model_kwargs`: Additional arguments passed to the model initialization
  - `torch_dtype`: Specifies the data type for model weights (bfloat16 for better memory efficiency)
- `device_map`: Automatically handles device placement (CPU/GPU)

In [ ]:
pipe = pipeline(
    'text-generation',
    model=model_id,
    model_kwargs={'torch_dtype': torch.bfloat16},
    device_map='auto'
)

## 2️⃣ Setting Up Streaming

We use `TextIteratorStreamer` to enable token-by-token streaming of the model's output.
Key parameters:

- `tokenizer`: The model's tokenizer for decoding tokens
- `skip_prompt`: Skip the input prompt in the output
- `skip_special_tokens`: Skip special tokens like <|endoftext|> in the output

### How the Streamer Works
1. Creates a queue to receive tokens
2. Registering a callback with the model
3. Decoding tokens as they arrive
4. Yielding decoded text to the consumer

In [ ]:
streamer = TextIteratorStreamer(
    pipe.tokenizer, 
    skip_prompt=True, 
    skip_special_tokens=False
)

## 3️⃣ Preparing the Chat

We format our conversation using the chat template format. Each message has:
- `role`: The speaker's role (system, user, assistant)
- `content`: The message content

The system message sets the model's behavior, while the user message contains the query.

### Chat Template Processing
1. Adds special tokens for message boundaries
2. Formats roles and content
3. Adds generation prompts
4. Ensures proper tokenization

In [ ]:
messages = [
    {'role': 'system', 'content': 'You are a pirate chatbot who always responds in pirate speak!'},
    {'role': 'user', 'content': 'What are black holes?'},
]

## 4️⃣ Generation Parameters

We configure the text generation with several important parameters:

| Parameter | Description |
|-----------|-------------|
| `text_inputs` | The formatted chat messages |
| `streamer` | Our streaming handler |
| `max_new_tokens` | Maximum number of tokens to generate |
| `do_sample` | Enable sampling for more diverse outputs |
| `temperature` | Controls randomness (higher = more creative, lower = more focused) |
| `top_p` | Nucleus sampling parameter (controls diversity of word choices) |

### Generation Process
1. Tokenizes the input
2. Creates attention masks
3. Runs the model forward pass
4. Samples from the output distribution
5. Updates the input sequence
6. Repeats until max tokens or stop condition

In [ ]:
generation_kwargs = {
    'text_inputs': messages,
    'streamer': streamer,
    'max_new_tokens': 2048,
    'do_sample': True,
    'temperature': 0.7,
    'top_p': 0.95,    
}

## 5️⃣ Running the Generation

We run the generation in a separate thread to enable streaming. This allows us to:
1. Start the generation process
2. Print tokens as they're generated
3. Maintain a responsive interface

The `Thread` class from Python's threading module handles this asynchronous execution.

### Threading Process
1. Main thread: Sets up the streamer and starts generation
2. Worker thread: Runs the model generation
3. Streamer: Bridges between threads, handling token queue
4. Main thread: Consumes and prints tokens as they arrive

In [ ]:
thread = Thread(target=pipe, kwargs=generation_kwargs)
thread.start()

# Print the generated text as it comes in
for text in streamer:
    print(text, end='', flush=True)

## 🧪 Try Different Queries

Feel free to modify the `messages` variable to explore different prompts and system messages.
Here are some ideas:

```python
# Example 1: Different system prompt and query
messages = [
    {'role': 'system', 'content': 'You are a helpful assistant who explains complex topics simply.'},
    {'role': 'user', 'content': 'Explain quantum entanglement like I\'m five.'},
]

# Example 2: Multi-turn conversation
messages = [
    {'role': 'system', 'content': 'You are a friendly chatbot.'},
    {'role': 'user', 'content': 'What\'s the weather like today?'},
    # Imagine the model responds, then you add the next turn:
    # {'role': 'assistant', 'content': 'It\'s sunny and warm!'},
    # {'role': 'user', 'content': 'Great! Any recommendations for outdoor activities?'},
]

# Example 3: Creative writing prompt
messages = [
    {'role': 'system', 'content': 'You are a famous fantasy author.'},
    {'role': 'user', 'content': 'Write the opening paragraph of a story about a dragon who loves baking.'},
]
```
Remember to re-run the cells after changing the `messages` to see the new output.

## 🔄 Understanding the Process

### Step-by-Step Overview

1. **Model Loading**: The pipeline loads the model and tokenizer, handling all the necessary
   initialization and device placement.

2. **Streaming Setup**: The `TextIteratorStreamer` creates a queue that receives tokens as they're
   generated, allowing us to process them in real-time.

3. **Message Formatting**: The chat template converts our messages into the format the model expects,
   including special tokens and role markers.

4. **Generation**: The model generates text token by token, with sampling parameters controlling
   the creativity and diversity of the output.

5. **Output**: Tokens are streamed through the `TextIteratorStreamer`, decoded, and printed in
   real-time, creating a responsive chat experience.